# Performance Deep Dive

Understanding and optimizing performance with GROMOS-RS Python bindings.

## Topics
1. [Benchmarking Methodology](#benchmarking)
2. [Memory Profiling](#memory)
3. [SIMD Vectorization](#simd)
4. [Cache Effects](#cache)
5. [Scaling Analysis](#scaling)
6. [Optimization Tips](#optimization)

In [ ]:
import gromos
import numpy as np
import time
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

## 1. Benchmarking Methodology <a name="benchmarking"></a>

Proper benchmarking is essential for understanding performance.

In [ ]:
def benchmark(func, n_iterations=1000, n_warmup=100):
    """Benchmark a function with warmup"""
    # Warmup
    for _ in range(n_warmup):
        func()
    
    # Actual timing
    times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        func()
        times.append(time.perf_counter() - start)
    
    times = np.array(times)
    return {
        'mean': times.mean(),
        'std': times.std(),
        'min': times.min(),
        'max': times.max(),
        'median': np.median(times)
    }

def print_benchmark(name, results):
    """Print benchmark results"""
    print(f"{name}:")
    print(f"  Mean:   {results['mean']*1e6:>8.2f} μs")
    print(f"  Median: {results['median']*1e6:>8.2f} μs")
    print(f"  Std:    {results['std']*1e6:>8.2f} μs")
    print(f"  Min:    {results['min']*1e6:>8.2f} μs")
    print(f"  Max:    {results['max']*1e6:>8.2f} μs")

In [ ]:
# Benchmark Vec3 operations
v1 = gromos.Vec3(1.0, 2.0, 3.0)
v2 = gromos.Vec3(4.0, 5.0, 6.0)

operations = {
    'Addition': lambda: v1 + v2,
    'Subtraction': lambda: v1 - v2,
    'Dot product': lambda: v1.dot(v2),
    'Cross product': lambda: v1.cross(v2),
    'Length': lambda: v1.length(),
    'Distance': lambda: v1.distance(v2),
    'Normalize': lambda: v1.normalize(),
}

print("Vec3 Operations Benchmark (10,000 iterations):\n")
for name, func in operations.items():
    results = benchmark(func, n_iterations=10000)
    print_benchmark(name, results)
    print()

## 2. Memory Profiling <a name="memory"></a>

Understanding memory usage and allocation patterns.

In [ ]:
import sys

def memory_usage(obj):
    """Get memory usage of an object"""
    if isinstance(obj, np.ndarray):
        return obj.nbytes
    return sys.getsizeof(obj)

# Test different system sizes
sizes = [10, 100, 1000, 10000, 100000]

print("Memory Usage by System Size:\n")
print(f"{'N atoms':<12} {'State (KB)':<15} {'Positions (KB)':<15} {'Total (KB)':<15}")
print("-" * 60)

for n in sizes:
    state = gromos.State(num_atoms=n, num_temp_groups=1, num_energy_groups=1)
    
    # Set positions
    pos = np.random.rand(n, 3).astype(np.float32)
    state.set_positions(pos)
    
    # Get positions back
    retrieved = state.positions()
    
    state_mem = sys.getsizeof(state)
    pos_mem = retrieved.nbytes
    total_mem = state_mem + pos_mem
    
    print(f"{n:<12} {state_mem/1024:<15.2f} {pos_mem/1024:<15.2f} {total_mem/1024:<15.2f}")

In [ ]:
# Memory layout visualization
n = 5
state = gromos.State(num_atoms=n, num_temp_groups=1, num_energy_groups=1)
pos = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]], dtype=np.float32)
state.set_positions(pos)

retrieved = state.positions()

print("Memory Layout Analysis:\n")
print(f"Array shape:     {retrieved.shape}")
print(f"Array strides:   {retrieved.strides} bytes")
print(f"Element size:    {retrieved.itemsize} bytes")
print(f"Total size:      {retrieved.nbytes} bytes")
print(f"Contiguous:      {retrieved.flags['C_CONTIGUOUS']}")
print(f"Aligned:         {retrieved.flags['ALIGNED']}")
print(f"\nMemory address:  {retrieved.__array_interface__['data'][0]:x}")

## 3. SIMD Vectorization <a name="simd"></a>

Demonstrating SIMD performance benefits.

In [ ]:
# Compare SIMD vs scalar implementations
n_iterations = 100000

# SIMD (Vec3)
v1 = gromos.Vec3(1.0, 2.0, 3.0)
v2 = gromos.Vec3(4.0, 5.0, 6.0)

# NumPy (can use SIMD internally)
arr1 = np.array([1.0, 2.0, 3.0], dtype=np.float32)
arr2 = np.array([4.0, 5.0, 6.0], dtype=np.float32)

# Pure Python (scalar)
def python_dot(a, b):
    return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

py_list1 = [1.0, 2.0, 3.0]
py_list2 = [4.0, 5.0, 6.0]

print("Dot Product Performance Comparison:\n")

# Vec3 (Rust SIMD)
start = time.time()
for _ in range(n_iterations):
    result = v1.dot(v2)
vec3_time = time.time() - start
print(f"Vec3 (Rust SIMD):  {vec3_time:.4f}s")

# NumPy
start = time.time()
for _ in range(n_iterations):
    result = np.dot(arr1, arr2)
numpy_time = time.time() - start
print(f"NumPy:             {numpy_time:.4f}s")

# Pure Python
start = time.time()
for _ in range(n_iterations):
    result = python_dot(py_list1, py_list2)
python_time = time.time() - start
print(f"Pure Python:       {python_time:.4f}s")

print(f"\nSpeedups:")
print(f"  Vec3 vs NumPy:     {numpy_time/vec3_time:.2f}×")
print(f"  Vec3 vs Python:    {python_time/vec3_time:.2f}×")
print(f"  NumPy vs Python:   {python_time/numpy_time:.2f}×")

In [ ]:
# Visualize speedups
methods = ['Pure\nPython', 'NumPy', 'Vec3\n(Rust SIMD)']
times = [python_time, numpy_time, vec3_time]
speedups = [python_time/t for t in times]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Execution time
ax1.bar(methods, times, color=['red', 'blue', 'green'], alpha=0.7, edgecolor='black')
ax1.set_ylabel('Time (seconds)')
ax1.set_title('Dot Product Performance\n(100,000 iterations)')
ax1.set_ylim(0, max(times) * 1.1)
for i, (method, t) in enumerate(zip(methods, times)):
    ax1.text(i, t + 0.01, f'{t:.3f}s', ha='center', va='bottom', fontweight='bold')

# Speedup
ax2.bar(methods, speedups, color=['red', 'blue', 'green'], alpha=0.7, edgecolor='black')
ax2.set_ylabel('Speedup (relative to Python)')
ax2.set_title('Relative Performance')
ax2.axhline(y=1, color='black', linestyle='--', linewidth=1)
for i, (method, s) in enumerate(zip(methods, speedups)):
    ax2.text(i, s + 0.5, f'{s:.1f}×', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

## 4. Cache Effects <a name="cache"></a>

Understanding cache behavior and memory access patterns.

In [ ]:
# Test cache effects with different access patterns
def sequential_access(arr):
    """Sequential memory access (cache-friendly)"""
    total = 0.0
    for i in range(len(arr)):
        total += arr[i, 0]  # Access in order
    return total

def random_access(arr, indices):
    """Random memory access (cache-unfriendly)"""
    total = 0.0
    for i in indices:
        total += arr[i, 0]  # Random access
    return total

# Test with different sizes
sizes = [100, 1000, 10000, 100000]
seq_times = []
rand_times = []

print("Cache Effects (Sequential vs Random Access):\n")
print(f"{'Size':<12} {'Sequential (ms)':<20} {'Random (ms)':<20} {'Ratio':<10}")
print("-" * 62)

for n in sizes:
    arr = np.random.rand(n, 3).astype(np.float32)
    indices = np.random.permutation(n)
    
    # Sequential
    start = time.time()
    for _ in range(100):
        sequential_access(arr)
    seq_time = (time.time() - start) * 1000
    
    # Random
    start = time.time()
    for _ in range(100):
        random_access(arr, indices)
    rand_time = (time.time() - start) * 1000
    
    seq_times.append(seq_time)
    rand_times.append(rand_time)
    
    ratio = rand_time / seq_time
    print(f"{n:<12} {seq_time:<20.2f} {rand_time:<20.2f} {ratio:<10.2f}")

print("\nConclusion: Random access is slower due to cache misses!")

In [ ]:
# Visualize cache effects
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(sizes))
width = 0.35

ax.bar(x - width/2, seq_times, width, label='Sequential', color='green', alpha=0.7)
ax.bar(x + width/2, rand_times, width, label='Random', color='red', alpha=0.7)

ax.set_xlabel('Array Size')
ax.set_ylabel('Time (ms)')
ax.set_title('Cache Effects: Sequential vs Random Access')
ax.set_xticks(x)
ax.set_xticklabels([f'{s:,}' for s in sizes])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Scaling Analysis <a name="scaling"></a>

How performance scales with system size.

In [ ]:
# Test scaling of operations
sizes = [10, 50, 100, 500, 1000, 5000, 10000, 50000]

create_times = []
set_times = []
get_times = []

print("Scaling Analysis:\n")
print(f"{'N atoms':<12} {'Create (μs)':<15} {'Set pos (μs)':<15} {'Get pos (μs)':<15}")
print("-" * 57)

for n in sizes:
    # Create state
    start = time.perf_counter()
    state = gromos.State(num_atoms=n, num_temp_groups=1, num_energy_groups=1)
    create_time = (time.perf_counter() - start) * 1e6
    
    # Set positions
    pos = np.random.rand(n, 3).astype(np.float32)
    start = time.perf_counter()
    state.set_positions(pos)
    set_time = (time.perf_counter() - start) * 1e6
    
    # Get positions
    start = time.perf_counter()
    retrieved = state.positions()
    get_time = (time.perf_counter() - start) * 1e6
    
    create_times.append(create_time)
    set_times.append(set_time)
    get_times.append(get_time)
    
    print(f"{n:<12} {create_time:<15.2f} {set_time:<15.2f} {get_time:<15.2f}")

In [ ]:
# Plot scaling
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(sizes, create_times, 'o-', label='Create', linewidth=2)
ax1.plot(sizes, set_times, 's-', label='Set positions', linewidth=2)
ax1.plot(sizes, get_times, '^-', label='Get positions', linewidth=2)
ax1.set_xlabel('Number of atoms')
ax1.set_ylabel('Time (μs)')
ax1.set_title('Operation Scaling (Linear)')
ax1.legend()
ax1.grid(alpha=0.3)

# Log scale
ax2.loglog(sizes, create_times, 'o-', label='Create', linewidth=2)
ax2.loglog(sizes, set_times, 's-', label='Set positions', linewidth=2)
ax2.loglog(sizes, get_times, '^-', label='Get positions', linewidth=2)

# Add O(n) reference line
n_ref = np.array([10, 50000])
t_ref = n_ref * 0.1  # Scaled for visibility
ax2.loglog(n_ref, t_ref, 'k--', label='O(n) reference', alpha=0.5)

ax2.set_xlabel('Number of atoms')
ax2.set_ylabel('Time (μs)')
ax2.set_title('Operation Scaling (Log-Log)')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Create: Nearly constant time (good!)")
print("- Set positions: Linear scaling O(n)")
print("- Get positions: Very fast, minimal overhead")

## 6. Optimization Tips <a name="optimization"></a>

Practical tips for optimizing your code.

### Tip 1: Batch Operations

Process multiple items at once instead of one at a time.

In [ ]:
# Bad: Process one at a time
def process_individual(n):
    vectors = [gromos.Vec3(i, i+1, i+2) for i in range(n)]
    total = 0.0
    for v in vectors:
        total += v.length()
    return total

# Good: Use NumPy arrays
def process_batch(n):
    arr = np.arange(n*3).reshape(n, 3).astype(np.float32)
    lengths = np.sqrt((arr**2).sum(axis=1))
    return lengths.sum()

n = 1000

start = time.time()
result1 = process_individual(n)
time1 = time.time() - start

start = time.time()
result2 = process_batch(n)
time2 = time.time() - start

print(f"Individual processing: {time1:.4f}s")
print(f"Batch processing:      {time2:.4f}s")
print(f"Speedup:               {time1/time2:.2f}×")

### Tip 2: Minimize Copies

Avoid unnecessary data copying.

In [ ]:
n = 10000
state = gromos.State(num_atoms=n, num_temp_groups=1, num_energy_groups=1)
pos = np.random.rand(n, 3).astype(np.float32)
state.set_positions(pos)

# Bad: Multiple copies
start = time.time()
for _ in range(100):
    p = state.positions().copy()  # Unnecessary copy!
    result = p.mean()
time_copy = time.time() - start

# Good: Use view directly
start = time.time()
for _ in range(100):
    p = state.positions()  # View, no copy
    result = p.mean()
time_view = time.time() - start

print(f"With copy:    {time_copy:.4f}s")
print(f"Without copy: {time_view:.4f}s")
print(f"Speedup:      {time_copy/time_view:.2f}×")

### Tip 3: Choose Right Data Type

float32 is usually sufficient and 2× faster than float64.

In [ ]:
n = 100000

# float32
arr32 = np.random.rand(n, 3).astype(np.float32)
start = time.time()
for _ in range(10):
    result = np.sqrt((arr32**2).sum(axis=1)).sum()
time32 = time.time() - start

# float64
arr64 = np.random.rand(n, 3).astype(np.float64)
start = time.time()
for _ in range(10):
    result = np.sqrt((arr64**2).sum(axis=1)).sum()
time64 = time.time() - start

print(f"float32: {time32:.4f}s ({arr32.nbytes/1024:.1f} KB)")
print(f"float64: {time64:.4f}s ({arr64.nbytes/1024:.1f} KB)")
print(f"Speedup: {time64/time32:.2f}×")
print(f"Memory:  {arr64.nbytes/arr32.nbytes:.1f}× more for float64")

### Tip 4: Preallocate Arrays

Reuse arrays instead of allocating new ones.

In [ ]:
n = 1000
iterations = 1000

# Bad: Allocate each time
start = time.time()
for _ in range(iterations):
    arr = np.zeros((n, 3), dtype=np.float32)  # New allocation
    arr[:] = np.random.rand(n, 3)
time_alloc = time.time() - start

# Good: Preallocate and reuse
arr = np.zeros((n, 3), dtype=np.float32)
start = time.time()
for _ in range(iterations):
    arr[:] = np.random.rand(n, 3)  # Reuse
time_reuse = time.time() - start

print(f"With allocation: {time_alloc:.4f}s")
print(f"With reuse:      {time_reuse:.4f}s")
print(f"Speedup:         {time_alloc/time_reuse:.2f}×")

## Summary

### Performance Best Practices:

1. **Use SIMD operations** (Vec3 methods)
2. **Batch process with NumPy** for arrays
3. **Minimize data copies** (use views)
4. **Choose float32** unless you need float64 precision
5. **Preallocate arrays** and reuse
6. **Sequential access** is cache-friendly
7. **Profile before optimizing** (measure, don't guess)

### Typical Speedups:
- Rust SIMD vs Python: **50-70×**
- Rust SIMD vs NumPy: **2-5×** (small vectors)
- Batch vs individual: **10-100×**
- Zero-copy vs copying: **2-10×**

### Remember:
- **Rust core** handles hot loops
- **Python** handles high-level logic
- **NumPy** for array operations
- **Profile** to find bottlenecks

The Polars architecture gives you the best of all worlds!